# Atmospheric and Topographic Correction for Sentinel-2 MSI: MultiSpectral Instrument, Level-1C
Sentinel-2 2A imagery in Google Earth Engine was processed on a tile by tile basis so that atmospheric corrections are inconsistent from tile to tile. Anisotropy was also not well corrected for. To generate consistent mosaics, we adapted the approach described in Potapov et al. (2012) that uses long term averages of MODIS reflectance as a target to debias sentinel imagery at the scene level and across track (to correct for anisotropy). MODIS does not contain an analog for Sentinel's nir-2 band. Sentinel's nir-2 band was therefore corrected using the nearest MODIS band and should be used with caution. Topographic correction was done with the Sun-Canopy-Sensor + C (SCSc) using functions created by Matt Macander and Pat Burns.
### Authors
Sam Jantz  
Patrick Jantz  
Camilo Fagua  
Pat Burns
#### Date
May 2022
#### Contact
Patrick.Jantz@nau.edu
#### Refs
Potapov, P.V., Turubanova, S.A., Hansen, M.C., Adusei, B., Broich, M., Altstatt, A., Mane, L. and Justice, C.O., 2012. Quantifying forest cover loss in Democratic Republic of the Congo, 2000–2010, with Landsat ETM+ data. Remote Sensing of Environment, 122, pp.106-116.

### Imports

In [1]:
import ee
import geemap
import sys, os, math
import geopandas as gpd
#import ipywidgets as widgets
#from ipyleaflet import WidgetControl
#sys.path.append('C:/Users/pj276/Projects/nasa_geo_colombia/code/sentinel2corrections')
#import sentinel_functions as sfx

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AZEOvhXxWoiezOewOlmcqO3o06mi_uvNhxEZVJrHGrIOTLxc8g5QIYLYUGE



Successfully saved authorization token.


In [4]:
# Test connection
ac = ee.ImageCollection('COPERNICUS/S2')
ac.first().date().getInfo()

{'type': 'Date', 'value': 1435400731456}

### Functions

In [5]:
# Functions
def getqa60LandMask(image):
    # Get sentinel qa60 60m cloudmask band
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Find pixels where both cirrus and opaque cloud are zero (not cloudy) in QA60
    # Set these pixels to 1
    # Set all other values to 0
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('qa60mask')
    # Add land mask
    lMask = image.select('SCL')
    mask2 = lMask.eq(4).Or(lMask.eq(5)).Or(lMask.eq(255)).rename('landmask')
    return image.addBands([mask, mask2])

def pctCloudLand(image):
    # Get image footprint
    geom = image.geometry()
    # Intersect with tilesamp
    fint = targetTile.geometry().intersection(geom)
    # Calculate percent of intersection that is cloudy
    # Clouds
    cSum = image.select('qa60mask').eq(0).reduceRegion(ee.Reducer.sum(),geometry=fint,scale=SCALE).getNumber('qa60mask')
    # No clouds
    ncSum = image.select('qa60mask').eq(1).reduceRegion(ee.Reducer.sum(),geometry=fint,scale=SCALE).getNumber('qa60mask')
    # Cloud + no cloud area
    cncTotal = cSum.add(ncSum)
    # Calc percent
    cPct = (cSum.divide(cncTotal)).multiply(100)
    # Calculate percent of intersection that is land
    # Land
    lSum = image.select('landmask').eq(1).reduceRegion(ee.Reducer.sum(),geometry=fint,scale=SCALE).getNumber('landmask')
    # No land 
    nlSum = image.select('landmask').eq(0).reduceRegion(ee.Reducer.sum(),geometry=fint,scale=SCALE).getNumber('landmask')
    # Land + no land area
    lnlTotal = lSum.add(nlSum)
    # Calc percent
    lPct = (lSum.divide(lnlTotal)).multiply(100)
    # Get percent of tile  covered by valid image footprint
    #tPct = (fint.area().divide(targetTile.geometry().area())).multiply(100)
    # Add propery to image
    #return image.set({'cPct':cPct, 'tPct':tPct})
    return image.set({'cPct':cPct, 'lPct':lPct})

# From get_parameters script
# Construct collection, filtering on date, cloud cover
def get_s2_cld_col(aoi, dFilter):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')\
        .filterBounds(aoi)\
        .filter(dFilter)\
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
    # Add an additional filter that considers cloud cover within the intersection of the valid image footprint
    # and the tile.Only select SCL band.
    s2_sr_col = s2_sr_col.map(getqa60LandMask).map(pctCloudLand)\
        .filter(ee.Filter.lte('cPct', CLOUD_FILTER))\
        .filter(ee.Filter.gt('lPct', 0.1)).select(['SCL'])
        #.filter(ee.Filter.gte('tPct', 10)).select(['SCL'])
    
    # Import and filter S2 TOA
    s2_col = (ee.ImageCollection('COPERNICUS/S2')\
        .filterBounds(aoi)\
        .filter(dFilter))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filter(dFilter))
    
    # Add SCL to TOA as property
    col = ee.ImageCollection(ee.Join.saveFirst('scl').apply(**{
        'primary': s2_col,
        'secondary': s2_sr_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
            })
        })) 

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

# Calculate parameters for bias correction
def get_parameters(img):
    #Get other images from the same datastrip and latitude
    dstrip = img.get('DATASTRIP_ID')
    s2col = ee.ImageCollection('COPERNICUS/S2').filter(ee.Filter.eq('DATASTRIP_ID',dstrip))
    s2col = s2col.map(uly)
    img= uly(img)
    s2col = s2col.filter(ee.Filter.eq('uly',img.get('uly')))
    sza = s2col.aggregate_mean('MEAN_SOLAR_ZENITH_ANGLE')

    # Get bounds for sampling
    geom = s2col.geometry()
    bnds = ee.Geometry.Polygon(geom.bounds().coordinates())
    img_ids = s2col.toList(s2col.size())

    #Create distance map with 0 at central meridian
    crs = ee.Image(img_ids.get(0)).select('B2').projection().crs()
    prj = ee.Projection(crs)
    x = ee.Image.pixelCoordinates(prj).select('x').divide(img.constant(1000))
    mx = x.reduceRegion(reducer=ee.Reducer.max(),geometry=s2col.geometry(),
                        scale=120,maxPixels=1e13).getNumber('x')
    mn = x.reduceRegion(reducer=ee.Reducer.min(),geometry=s2col.geometry(),
                        scale=120,maxPixels=1e13).getNumber('x')
    offset = ee.Number.divide(mx.add(mn),2)
    
    #add offset
    dist = x.subtract(ee.Image.constant(offset))
    scl = ee.Image(img.get('scl'))

    # Get S2SR with same data strip ID and latitude coordinate as TOA
    s2srcol = ee.ImageCollection('COPERNICUS/S2_SR').filter(ee.Filter.eq('DATASTRIP_ID',
                                                                         scl.get('DATASTRIP_ID')))
    s2srcol = s2srcol.map(uly)
    scl = uly(scl)
    s2srcol = s2srcol.filter(ee.Filter.eq('uly',scl.get('uly')))

    # Add mask from SCL band and QA60 band
    diff = s2col.map(subtract_modis)
    diff = diff.combine(s2srcol.select(['SCL','QA60']))
    diff = diff.map(add_mask)
    diff = diff.select(['blue','green','red','re1','re2','re3','nir','nir2','swir1','swir2'])

    # Mosaic all tiles in swath and get initial estimate of scene bias
    mos = diff.mosaic()
    md = mos.reduceRegion(ee.Reducer.median(),geometry=bnds,scale=SCALE,bestEffort=True,tileScale=4)
    bias = ee.List([md.get('blue'),md.get('green'),md.get('red'),md.get('re1'),md.get('re2'),
                    md.get('re3'),md.get('nir'),md.get('nir2'),md.get('swir1'),md.get('swir2')])
    mos = mos.subtract(ee.Image.constant(bias))

    # 2nd mosaic to get final bias and across track slopes
    mos = mos.select(['blue','green','red','re1','re2','re3','nir','nir2','swir1','swir2']).rename('Mblue','Mgreen','Mred','Mre1','Mre2','Mre3','Mnir','Mnir2','Mswir1','Mswir2')
    s2mos = diff.mosaic()
    mos = mos.addBands([s2mos,dist,ee.Image.constant(1)])
    
    # Dicts to store median bias across track slopes
    slps = ee.Dictionary()
    b = ee.Dictionary()
    
    ###### Sample each band separately and get median bias and slope ######
    for band in bands:
        samp = mos.select(band,'M'+band,'x','constant').sample(region=bnds,scale=SCALE,factor=1.0,
                                                               dropNulls=True,tileScale=8)
        samp = samp.filter(ee.Filter.gte('M'+band,-500)).filter(ee.Filter.lte('M'+band,500))
        ols=samp.reduceColumns(ee.Reducer.linearRegression(numX=2,numY=1),selectors=['x','constant',band])
        slps = slps.set(band,ee.List(ols.getArray('coefficients').toList().get(0)).get(0))
        md = samp.reduceColumns(ee.Reducer.median(),selectors=[band])
        b = b.set(band, md.get('median'))
    m = ee.List([slps.get('blue'),slps.get('green'),slps.get('red'),slps.get('re1'),slps.get('re2'),
                 slps.get('re3'),slps.get('nir'),slps.get('nir2'),slps.get('swir1'),slps.get('swir2')])
    bias = ee.List([b.get('blue'),b.get('green'),b.get('red'),b.get('re1'),b.get('re2'),b.get('re3'),
                    b.get('nir'),b.get('nir2'),b.get('swir1'),b.get('swir2')])
    
    # Return all parameters as a feature
    dic = ee.Dictionary({'slopes': ee.String.encodeJSON(m),'biases':ee.String.encodeJSON(bias),'ID':img.get('ID'),
                         's2id':dstrip,'s2srid':scl.get('DATASTRIP_ID'),'ULY':img.get('uly'),'crs':crs,
                         'offset':ee.String.encodeJSON(offset),'sza':ee.String.encodeJSON(sza)})
    return(ee.Feature(img.geometry().centroid(),dic))

# Subtract MODIS normalization target
def subtract_modis(img):
    diff = img.select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']).rename('blue','green','red','re1','re2','re3','nir','nir2','swir1','swir2').subtract(modis)
    return(diff)

# Simple mask from SCL band and QA60 band
def add_mask(img):
    scl = img.select('SCL')
    # Select vegetation, bare soil, or 255 (not sure what that is)
    msk = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(255))
    # Select non-cloudy pixels
    qa = img.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Find pixels where both cirrus and cloud are zero (not cloudy) in QA60
    # Set these pixels to 1
    # Set all other values to 0
    msk2 = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('qa60mask')
    # Set pixels to 1 where there is bare soil/vegetation and no clouds
    return(img.updateMask(msk).updateMask(msk2))

# Function to get UL latitude coordinate 
def uly(img):
    listCoords = ee.Array.cat(img.geometry().coordinates(), 1)
    yCoords = listCoords.slice(1, 1, 2)
    yMax = yCoords.reduce('max', [0]).get([0,0])
    yMax = ee.Number(yMax).multiply(100).round()
    return(img.set({'uly':yMax}))

# Join datastrip ID and uly lat for unique swatch ID
def swath_id(img):
    spc = ee.String('_')
    uly = ee.Number(img.get('uly')).format("%d")
    ds_id = ee.String(img.get('DATASTRIP_ID'))
    img=img.set({'ID':ds_id.cat(spc.cat(uly))})
    return(img)

# From apply parameters

# Add cloud info
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = img.select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image(is_cloud))

# Add cloud shadows
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).    
    dark_pixels = img.select('nir').lt(NIR_DRK_THRESH).multiply(not_water).rename('dark_pixels')

    ## Determine the direction to project cloud shadow from clouds.
    #shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    
    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    
    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
                .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
                .select('distance').mask().rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image(shadows))

# Create cloud shadow mask
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))
    
    # Remove small cloud-shadow patches and dilate remaining pixels.
    #is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER)).rename('cloudmask')

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

# Add qaqc values
def add_qa(img):
    img_cld_shdw = add_cld_shdw_mask(img)
    scl = img.select('SCL')
    qa = scl.eq(1).Or(scl.eq(3)).Or(scl.eq(8)).Or(scl.eq(9)).Or(scl.eq(10))
    qa_mask = qa.add(img_cld_shdw.select('cloudmask')).gt(0).rename('qa_mask')
    # Find pixels where both cirrus and cloud are zero (not cloudy) in QA60
    # Set these pixels to 1
    # Set all other values to 0
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    qa60 = img.select('QA60')
    qa60mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('qa60mask')
    return img_cld_shdw.addBands([qa_mask,qa60mask])

# Apply qaqc values
def apply_qa(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('qa_mask').Not()
    # Clouds and cirrus from qa60 band are 0, else 1
    not_cld_cirrus = img.select('qa60mask')
    # Subset reflectance bands and update their masks, return the result.
    return img.select(['blue','green','red','re1','re2','re3','nir','nir2','swir1','swir2'])\
        .updateMask(not_cld_shdw).updateMask(not_cld_cirrus).addBands(not_cld_shdw)

# Apply parameters to reflectance
def apply_parameters(img):
    scl = ee.Image(img.get('scl'))
    clds = ee.Image(img.get('s2cloudless'))
    prms = ee.Feature(img.get('params'))
    az = ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE'))
    ze = ee.Number(img.get('MEAN_SOLAR_ZENITH_ANGLE'))
    t_start = img.get('system:time_start')
    m = ee.List(ee.String.decodeJSON(prms.get('slopes')))
    b = ee.List(ee.String.decodeJSON(prms.get('biases')))
    crs = prms.get('crs')
    offset = ee.Number(ee.String.decodeJSON(prms.get('offset')))
    prj = ee.Projection(crs)
    x = ee.Image.pixelCoordinates(prj).select('x').divide(img.constant(1000))
    dist = x.subtract(ee.Image.constant(offset))
    img = img.select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']).rename('blue','green','red','re1','re2','re3','nir','nir2','swir1','swir2')
    img = img.subtract(ee.Image.constant(b))
    at_bias = dist.multiply(ee.Image.constant(m))
    bias_adj = img.subtract(at_bias)
    bias_adj = bias_adj.set({'scl':scl, 'system:time_start':t_start, 'MEAN_SOLAR_AZIMUTH_ANGLE':az, 'MEAN_SOLAR_ZENITH_ANGLE':ze})
    return(bias_adj.addBands([scl,clds]))

# Calculate illumination
def illuminationCondition(img):
    # Extract image metadata about solar position
    SZ_rad = ee.Image.constant(ee.Number(img.get('MEAN_SOLAR_ZENITH_ANGLE'))).multiply(3.14159265359).divide(180).clip(img.geometry().buffer(10000))
    SA_rad = ee.Image.constant(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).multiply(3.14159265359).divide(180)).clip(img.geometry().buffer(10000))
    # Creat terrain layers
    slp = ee.Terrain.slope(dem).clip(img.geometry().buffer(10000))
    slp_rad = ee.Terrain.slope(dem).multiply(3.14159265359).divide(180).clip(img.geometry().buffer(10000))
    asp_rad = ee.Terrain.aspect(dem).multiply(3.14159265359).divide(180).clip(img.geometry().buffer(10000))

    # Calculate the Illumination Condition (IC)
    # slope part of the illumination condition
    cosZ = SZ_rad.cos()
    cosS = slp_rad.cos()
    slope_illumination = cosS.expression("cosZ * cosS", 
                                          {'cosZ': cosZ,
                                           'cosS': cosS.select('slope')})
    # aspect part of the illumination condition
    sinZ = SZ_rad.sin()
    sinS = slp_rad.sin()
    cosAziDiff = (SA_rad.subtract(asp_rad)).cos()
    aspect_illumination = sinZ.expression("sinZ * sinS * cosAziDiff", 
                                           {'sinZ': sinZ,
                                            'sinS': sinS,
                                            'cosAziDiff': cosAziDiff})
    # full illumination condition (IC)
    ic = slope_illumination.add(aspect_illumination)
    #// Add IC to original image
    img_plus_ic = ee.Image(img.addBands(ic.rename('IC')).addBands(cosZ.rename('cosZ')).addBands(cosS.rename('cosS')).addBands(slp.rename('slope')));
    return img_plus_ic

#////////////////////////////////////////////////////////////////////////////////
#// Function to apply the Sun-Canopy-Sensor + C (SCSc) correction method to each 
#// image. Function by Patrick Burns (pb463@nau.edu) and Matt Macander 
#// (mmacander@abrinc.com)
def illuminationCorrection(img):
    props = img.toDictionary()
    st = img.get('system:time_start')
    
    img_plus_ic = img
    mask1 = img_plus_ic.select('nir').gt(-0.1)
    mask2 = (img_plus_ic.select('slope').gte(5).And(img_plus_ic.select('IC').gte(0)).And(img_plus_ic.select('nir').gt(-0.1)))
    img_plus_ic_mask2 = ee.Image(img_plus_ic.updateMask(mask2))
    
    #// Specify Bands to topographically correct  
    bandList = ['blue','green','red','re1','re2','re3','nir','nir2','swir1','swir2']
    compositeBands = img.bandNames()
    nonCorrectBands = img.select(compositeBands.removeAll(bandList))
    
    #geom = ee.Geometry(img.get('system:footprint')).bounds().buffer(10000)
    
    def apply_SCSccorr(band):
        #method = 'SCSc'
        out =  (ee.Image(1).addBands(img_plus_ic_mask2.select('IC', band)).reduceRegion(reducer= ee.Reducer.linearRegression(2,1), geometry=ee.Geometry(img.geometry()), scale=scale, bestEffort=True, maxPixels=1e10))
        fit = out.combine({"coefficients": ee.Array([[1],[1]])}, False)
        #//Get the coefficients as a nested list, 
        #//Cast it to an array, and get just the selected column
        out_a = (ee.Array(fit.get('coefficients')).get([0,0]))
        out_b = (ee.Array(fit.get('coefficients')).get([1,0]))
        out_c = out_a.divide(out_b)
        
        #// Apply the SCSc correction
        SCSc_output = img_plus_ic_mask2.expression(
        "((image * (cosB * cosZ + cvalue)) / (ic + cvalue))", {
        'image': img_plus_ic_mask2.select(band),
        'ic': img_plus_ic_mask2.select('IC'),
        'cosB': img_plus_ic_mask2.select('cosS'),
        'cosZ': img_plus_ic_mask2.select('cosZ'),
        'cvalue': out_c
        })
      
        return SCSc_output

    #img_SCSccorr = ee.Image(bandList.map(apply_SCSccorr)).addBands(img_plus_ic.select('IC'))
    img_SCSccorr = ee.Image(list(map(apply_SCSccorr,bandList))).addBands(img_plus_ic.select('IC'))
    bandList_IC = ee.List([bandList, 'IC']).flatten()
    img_SCSccorr = img_SCSccorr.unmask(img_plus_ic.select(bandList_IC)).select(bandList)
    return img_SCSccorr.addBands(nonCorrectBands).setMulti(props).set('system:time_start',st)

# Apply topo correction
def topoCorrection(collection):
    collection2 = collection.map(illuminationCondition)
    collection3 = collection2.map(illuminationCorrection)
    return collection3

#------------------------------------------------------------
# Functions From Camilo's scripts
# Function to rename bands
def band_rename(img):
    return img.select(
    ['blue', 'green', 'red', 're1', 're2', 're3', 'nir', 'nir2', 'swir1', 'swir2', 'qa_mask'],
    ['BLUE','GREEN','RED','RED_EDGE_1','RED_EDGE_2','RED_EDGE_3','NIR','RED_EDGE_4','SWIR_1','SWIR_2','QA_MASK'])

# Functions to calculate and add indices
# function to calculate NDVI
def ndvi_calc(img):
    return img.normalizedDifference(['NIR', 'RED']).select([0],['NDVI'])

# Function to calculate EVI
def evi_calc(img):
    num = ee.Image((img.select(['NIR']).subtract(img.select(['RED']))).multiply(ee.Image(2.5)))
    den = ee.Image(img.select(['NIR']).add(img.select(['RED']).multiply(6.0)).subtract(img.select(['BLUE']).multiply(7.5)).add(1.0))
    return (num.divide(den)).select([0],['EVI'])


# Function to Soil Adjusted Vegetation Index (SAVI)
def savi_calc(img):
    num = ee.Image((img.select(['NIR']).subtract(img.select(['RED']))))
    den = ee.Image((img.select(['NIR']).add(img.select(['RED']).add(0.5))).multiply(1.5))
    return (num.divide(den)).select([0],['SAVI'])

# Function to spectral variability vegetation index (SVVI)
def svvi_calc(img):
    term1 = ee.Image(img.select(['BLUE','GREEN','RED','NIR','SWIR_1','SWIR_2'])).reduce(ee.Reducer.stdDev());
    term2 = ee.Image(img.select(['NIR','SWIR_1','SWIR_2'])).reduce(ee.Reducer.stdDev());
    return (term1.subtract(term2)).select([0],['SVVI'])

# Red-edge Normalized Difference Vegetation Index - RNDVI
def rndvi_calc(img):
    return img.normalizedDifference(['NIR', 'RED_EDGE_2']).select([0],['RNDVI'])

# Function to add indices to an image
# NDVI, EVI, SAVI, SVVI, RNDVI
def addIndices(in_image):
    temp_image = in_image.float().divide(10000.0)
    return in_image.addBands(ee.Image(ndvi_calc(in_image)).multiply(10000.0).int16())\
        .addBands(ee.Image(evi_calc(temp_image)).multiply(10000.0).int16())\
        .addBands(ee.Image(savi_calc(temp_image)).multiply(10000.0).int16())\
        .addBands(ee.Image(svvi_calc(temp_image)).multiply(10000.0).int16())\
        .addBands(ee.Image(rndvi_calc(temp_image)).multiply(10000.0).int16())

# Function texture
def texture(img):
    img_16bit = img.int16()
    return img_16bit.glcmTexture(size=3)

### Map

In [6]:
Map = geemap.Map(center=(4, -74), zoom=6)
Map

Map(center=[4, -74], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [7]:
# Path to store sentinel correction parameters
# Append the image cloud threshold used to calculate params
sparamPath = 'projects/ee-jantzenator/assets/sentinel_acorr/params'

# Tile file path
#tileFile = r"G:\My Drive\Projects\nasa_geo_colombia\geodata\camilo_assets\GRID_COLOMBIA.shp"
tiles = ee.FeatureCollection('users/jf2238/GRID_COLOMBIA')

# Map tiles
Map.addLayer(tiles, {}, 'Tiles')

# Modis composite used as target
modis = ee.Image('users/sjantzterp/modis2senv3')

SCALE = 120 #Scale in meters for sampling. Memory errors more likely under 120m

# Part 1
### Calculate Parameters

In [7]:
# Params
# Sentinel bands
bands = ['blue','green','red','re1','re2','re3','nir','nir2','swir1','swir2']

# Params
CLOUD_FILTER = 45 # Scene level cloud filtering. Take scenes with cloud percentages less than this


# Start and end year/month
#sYear=2017
#eYear=2022
#sMonth=1
#eMonth=12

# Date filter object
dFilter = ee.Filter.date('2017-01-01','2022-12-31')
#print(dFilter.getInfo())



In [8]:
# Loop through tiles and calculate parameters
#for i in range(0,230):
#for i in range(1,230):
#for i in [64,159,174,209,210,215,220,221,222,227,228,229]:
for i in [209]:
    # Tile id
    # Fid
    Fid = i

    # Map selected tile
    targetTile = tiles.filter(ee.Filter.eq('FID_STRING', str(Fid)))
    Map.addLayer(targetTile, {}, 'Target Tile')
    
    # Get collection
    s2_col = get_s2_cld_col(targetTile, dFilter)
    s2_col = s2_col.map(uly).map(swath_id)
    # Remove duplicates in swaths
    s2_col = ee.ImageCollection(s2_col.distinct('ID'))
    print('Collection size: ', s2_col.size().getInfo())

    # Get parameters and export to asset
    fc = ee.FeatureCollection(s2_col.map(get_parameters))
    task = ee.batch.Export.table.toAsset(collection=fc,description='params45_' + str(Fid),
                                         assetId= sparamPath + '/params45_' + str(Fid))
    task.start()

Collection size:  94


In [10]:
# last task status
# Easier to check at https://code.earthengine.google.com/tasks
print(task.status())

{'state': 'RUNNING', 'description': 'params45_209', 'creation_timestamp_ms': 1655600444529, 'update_timestamp_ms': 1655600448137, 'start_timestamp_ms': 1655600448106, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'V2MPZNX7JHVPS6TDIJLXDEMO', 'name': 'projects/earthengine-legacy/operations/V2MPZNX7JHVPS6TDIJLXDEMO'}


# -------------------------------------------------------------------------------------------
# Part 2
### Atmospheric and Topo Correction

In [8]:
#/////////////////////////////// I) FIRST MOSAIC 2019 -2020/////////////////////////////
# Apply corrections to imagery
CLOUD_FILTER = 45
CLD_PRB_THRESH = 20 # pixel level cloud probability threshold
NIR_DRK_THRESH = 1500 #unscaled
CLD_PRJ_DIST = 1 # Code multiplies this by 10 (so I guess that's 20 pixels?)
BUFFER = 50 #in pixels (but may be meters?)

In [19]:
# Loop throught and export mosaics
# Max range is 230
# Params 209 not found
# So run 144,209, then 210,230
# Up to 144 is essentially the bottom half of the country
# and includes most of the Colombian Amazon
#for j in range(144,209):
#for j in [63, 211, 216, 217, 218, 223, 224, 225, 226, 229]:
#for j in list([27]+ list(range(29,209))+list(range(210,230))):
# mean: 0, 43, 87, 113, 211, 214
# sd: 0, 89, 211
# savg: 28, 41, 62, 126, 216, 219, 222, 223, 224, 225, 226, 227, 228, 229
# dvar: None

# Export flags
e1 = False # mean
e2 = False # sd
e3 = False # savg
e4 = True # dvar
    
#for j in list(list(range(0,209))+list(range(210,230))):
#for j in [28, 41, 62, 126, 216, 219, 222, 223, 224, 225, 226, 227, 228, 229]:
for j in [210]:
    Fid = j

    #Start and end year/month
    #sYear=2019
    #eYear=2020
    #sMonth=1
    #eMonth=12

    # Mosaic number (either 1 or 2)
    mNumber = 2

    # Create date filter object and mosaic output name
    if mNumber == 1:
        # Date filter object for 1st Mosaic
        dFilter = ee.Filter.date('2019-01-01','2020-12-31')
        # Date string
        dString = '2019_2020'
    elif mNumber == 2:
        # Date filter object for 2nd Mosaic
        d1 = ee.Filter.date('2017-01-01','2018-12-31')
        d2 = ee.Filter.date('2021-01-01','2022-12-31')
        dFilter = ee.Filter.Or(d1, d2)
        dString = '2017_18_2021_22'

    targetTile = tiles.filter(ee.Filter.eq('FID_STRING', str(Fid)))
    params_fc = ee.FeatureCollection('projects/ee-jantzenator/assets/sentinel_acorr/params/params45_'+str(Fid))
    s2_col = get_s2_cld_col(targetTile, dFilter)
    s2_col = s2_col.map(uly).map(swath_id)

    col = ee.ImageCollection(ee.Join.saveFirst('params').apply(**{
        'primary': s2_col,
        'secondary': params_fc,
        'condition': ee.Filter.equals(**{
            'leftField': 'ID',
            'rightField': 'ID'
            })
        }))

    #------------------------------------------------------------
    # Atmospheric correction
    s2fin = col.map(apply_parameters).map(add_qa).map(apply_qa)
    # Subset images for tile 226 (due to size issues). Use size of largest image collection
    # that ran successfully
    print(s2fin.size().getInfo())
    if Fid == 226:
        s2fin = ee.ImageCollection(s2fin.randomColumn('random',13).sort('random').limit(688))
    print(s2fin.size().getInfo())
    
    #------------------------------------------------------------
    # Illumination params
    scale = 100
    toaOrSR = 'SR'
    #// get terrain layers
    dem = ee.Image("USGS/SRTMGL1_003")
    degree2radian = 0.01745

    # Topographic correction
    s2tc = topoCorrection(s2fin)
    print(s2tc.size().getInfo())

    # Rename bands, add indices
    s2tc = s2tc\
        .map(band_rename)\
        .map(addIndices)

    # MEAN and SD
    # Mean of spectral metrics
    mosaic_mean = s2tc.select(['BLUE','GREEN','RED','RED_EDGE_1','RED_EDGE_2','RED_EDGE_3','NIR','RED_EDGE_4','SWIR_1','SWIR_2',
                                  'NDVI', 'EVI', 'SAVI', 'SVVI','RNDVI']).mean().int16()
    # SD of spectral metrics
    mosaic_sd = s2tc.select(['BLUE','GREEN','RED','RED_EDGE_1','RED_EDGE_2','RED_EDGE_3','NIR','RED_EDGE_4','SWIR_1','SWIR_2',
                                  'NDVI', 'EVI', 'SAVI', 'SVVI','RNDVI'])\
                                  .reduce(ee.Reducer.stdDev()).int16()

    # TEXTURE
    # Calculate texture metrics
    s2tc_texture = s2tc.map(texture)
    # Mean of structure metrics
    TEXTURA_savg = s2tc_texture.select(['BLUE_savg','GREEN_savg','RED_savg','RED_EDGE_1_savg','RED_EDGE_2_savg',
                                             'RED_EDGE_3_savg','NIR_savg','RED_EDGE_4_savg','SWIR_1_savg','SWIR_2_savg',
                                             'NDVI_savg', 'EVI_savg', 'SAVI_savg', 'SVVI_savg','RNDVI_savg']).mean().int16();                       
    TEXTURA_dvar = s2tc_texture.select(['BLUE_dvar','GREEN_dvar','RED_dvar','RED_EDGE_1_dvar','RED_EDGE_2_dvar',
                                              'RED_EDGE_3_dvar','NIR_dvar','RED_EDGE_4_dvar','SWIR_1_dvar','SWIR_2_dvar',
                                              'NDVI_dvar', 'EVI_dvar', 'SAVI_dvar', 'SVVI_dvar','RNDVI_dvar']).mean().int16()
 
    # EXPORT 1
    #SENT2_MEAN_COLOMBIA
    #'Sent2_mean'.concat('_', c)
    if e1:
        task = ee.batch.Export.image.toAsset(image=mosaic_mean,description='med_' + str(Fid) + '_' + dString,
                                              assetId='projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_' + str(Fid) + '_' + dString,
                                              scale = 25, region = targetTile.geometry(),
                                              maxPixels=1e13)
        task.start()
    
    # EXPORT 2
    #SENT2_SD_COLOMBIA
    #'Sent2_sd'.concat('_', c)
    if e2:
        task = ee.batch.Export.image.toAsset(image=mosaic_sd,description='sd_' + str(Fid) + '_' + dString,
                                              assetId='projects/ee-jantzenator/assets/sentinel_acorr/SENT2_SD_COLOMBIA/Sent2_sd_' + str(Fid) + '_' + dString,
                                              scale = 25, region = targetTile.geometry(),
                                              maxPixels=1e13)
        task.start()

    # EXPORT 3
    #SENT2_savg_COLOMBIA
    #'Sent2_savg'.concat('_', c)
    if e3:
        task = ee.batch.Export.image.toAsset(image=TEXTURA_savg,description='savg_' + str(Fid) + '_' + dString,
                                              assetId='projects/ee-jantzenator/assets/sentinel_acorr/SENT2_SAVG_COLOMBIA/Sent2_savg_' + str(Fid) + '_' + dString,
                                              scale = 25, region = targetTile.geometry(),
                                              maxPixels=1e13)
        task.start()    

    # EXPORT 4
    #SENT2_dvar_COLOMBIA
    #'Sent2_dvar'.concat('_', c)
    if e4:
        task = ee.batch.Export.image.toAsset(image=TEXTURA_dvar,description='dvar_' + str(Fid) + '_' + dString,
                                              assetId='projects/ee-jantzenator/assets/sentinel_acorr/SENT2_DVAR_COLOMBIA/Sent2_dvar_' + str(Fid) + '_' + dString,
                                              scale = 25, region = targetTile.geometry(),
                                              maxPixels=1e13)
        task.start()
    
    

133
133
133


In [17]:
# Check status
task.status()

{'state': 'RUNNING',
 'description': 'med_210_2019_2020',
 'creation_timestamp_ms': 1689785459898,
 'update_timestamp_ms': 1689789569617,
 'start_timestamp_ms': 1689785469864,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'A4SWNNS5YVZIF3UISGKBCDFN',
 'name': 'projects/earthengine-legacy/operations/A4SWNNS5YVZIF3UISGKBCDFN'}

In [254]:
# Add to map
s2finMosaic = ee.Image('projects/ee-jantzenator/assets/sentinel_acorr/med6_' + str(Fid) + '_2019_2020')
Map.addLayer(s2finMosaic, vizParams, 's2_' + str(Fid))

# Extra

In [84]:
# Display parameters
vizParams = {
  'bands': ['red', 'green', 'blue'],
  'min': 300,
  'max': 800,
  'gamma': 1
}
vizParams = {
  'bands': ['swir1', 'nir', 'red'],
  'min': 188,
  'max': 4103,
  'gamma': 1
}


In [244]:
# Map first image (no topo correction)
Map.addLayer(s2fin.first(), vizParams, 's2')
# Map median (no topo correction)
Map.addLayer(s2fin.median().clip(targetTile.geometry()), vizParams, 's2med')

In [189]:
# Print number of masked pixels
stats = s2fin.median().select('nir').mask().eq(0).reduceRegion(reducer=ee.Reducer.sum(),
                                                 scale=25,geometry=targetTile.geometry(),
                                                maxPixels=1e9
)
print(stats.getInfo())

{'nir': 11092030.8745098}


In [77]:
# Print image info
bandNames = s2fin.first().bandNames()
print('Number of bands: ', bandNames.size().getInfo())
bandNames = s2fin.first().bandNames()
print('Band names: ', bandNames.getInfo())
properties = s2fin.first().propertyNames()
print('Metadata properties: ', properties.getInfo())
date = ee.Date(s2fin.first().get('system:time_start'))
print("Formatted date", date.format('Y-M-d').getInfo())

Number of bands:  11
Band names:  ['blue', 'green', 'red', 're1', 're2', 're3', 'nir', 'nir2', 'swir1', 'swir2', 'qa_mask']
Metadata properties:  ['system:time_start', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'MEAN_SOLAR_ZENITH_ANGLE', 'scl', 'system:index', 'system:bands', 'system:band_names']
Formatted date 2019-1-12


In [204]:
# Map monthly images if needed
# Only works with non topo corrected images as the topo correction times out...
months = ee.List.sequence(1, 12)
#years = ee.List.sequence(2020,2021)
years = ee.List([2020])

def function1(y):
    def function2(m):
        return s2fin.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month')).median().set('month', m).set('year', y)
    return months.map(function2)

mmonth = ee.ImageCollection.fromImages(years.map(function1).flatten())

listOfImages = mmonth.toList(mmonth.size())
img1 = ee.Image(listOfImages.get(11))
print(img1.getInfo())
Map.addLayer(img1, vizParams, 'm1')

#listOfImages = ee.Image(mmonth.toList(mmonth.size()))
#timage = ee.Image(listOfImages.get(2))
#print(timage.getInfo())

#Map.addLayer(timage, vizParams, 'm1')

{'type': 'Image', 'bands': [], 'properties': {'month': 12, 'year': 2021, 'system:index': '11'}}


In [252]:
# Modis composite used as target
print(modis.getInfo())
vizParams = {
  'bands': ['swir1', 'nir', 'red'],
  'min': 188,
  'max': 4103,
  'gamma': 1
}
Map.addLayer(modis, vizParams, 'modis')

{'type': 'Image', 'bands': [{'id': 'blue', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10806, 9784], 'crs': 'EPSG:4326', 'crs_transform': [0.004491576420597608, 0, -89.35991288778939, 0, -0.004491576420597608, 16.811970542296844]}, {'id': 'green', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10806, 9784], 'crs': 'EPSG:4326', 'crs_transform': [0.004491576420597608, 0, -89.35991288778939, 0, -0.004491576420597608, 16.811970542296844]}, {'id': 'red', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10806, 9784], 'crs': 'EPSG:4326', 'crs_transform': [0.004491576420597608, 0, -89.35991288778939, 0, -0.004491576420597608, 16.811970542296844]}, {'id': 're1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10806, 9784], 'crs': 'EPSG:4326', 'crs_transform': [0.004491576420597608, 0, -89.35991288778939, 0

In [28]:
# Tile id
# 0-230
# Fid
Fid = 64
#Fid = 16


# Map selected tile
targetTile = tiles.filter(ee.Filter.eq('FID_STRING', str(Fid)))
Map.addLayer(targetTile, {}, 'Target Tile')

# Get collection
s2_col = get_s2_cld_col(targetTile, dFilter)
s2_col = s2_col.map(uly).map(swath_id)
# Remove duplicates in swaths
s2_col = ee.ImageCollection(s2_col.distinct('ID'))
print('Collection size: ', s2_col.size().getInfo())

Collection size:  115


In [35]:
badImage = ee.ImageCollection('COPERNICUS/S2_SR').filter(ee.Filter.eq('system:index','20181216T155219_20181216T155215_T17NPC')).first()
vizParams = {
  'bands': ['B9', 'B5', 'B4'],
  'min': 188,
  'max': 4103,
  'gamma': 1
}
Map.addLayer(badImage, vizParams, 'badimage')


In [36]:
properties = badImage.propertyNames()
print('Metadata properties: ', properties.getInfo())

lpct = badImage.get('lPct')
print(lpct.getInfo())

cpct = badImage.get('cPct')
print(cpct.getInfo())

Metadata properties:  ['DATATAKE_IDENTIFIER', 'AOT_RETRIEVAL_ACCURACY', 'SPACECRAFT_NAME', 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE', 'system:id', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A', 'CLOUD_SHADOW_PERCENTAGE', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'system:footprint', 'VEGETATION_PERCENTAGE', 'SOLAR_IRRADIANCE_B12', 'system:version', 'SOLAR_IRRADIANCE_B10', 'SENSOR_QUALITY', 'SOLAR_IRRADIANCE_B11', 'GENERATION_TIME', 'SOLAR_IRRADIANCE_B8A', 'FORMAT_CORRECTNESS', 'CLOUD_COVERAGE_ASSESSMENT', 'THIN_CIRRUS_PERCENTAGE', 'system:time_end', 'WATER_VAPOUR_RETRIEVAL_ACCURACY', 'system:time_start', 'DATASTRIP_ID', 'PROCESSING_BASELINE', 'SENSING_ORBIT_NUMBER', 'NODATA_PIXEL_PERCENTAGE', 'SENSING_ORBIT_DIRECTION', 'GENERAL_QUALITY', 'GRANULE_ID', 'REFLECTANCE_CONVERSION_CORRECTION', 'MEDIUM_PROBA_CLOUDS_PERCENTAGE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8', 'DATATAKE_TYPE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4', 'MEA

In [41]:
def getqa60LandMask2(image):
    # Get sentinel qa60 60m cloudmask band
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Find pixels where both cirrus and opaque cloud are zero (not cloudy) in QA60
    # Set these pixels to 1
    # Set all other values to 0
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0)).rename('qa60mask')
    # Add land mask
    lMask = image.select('SCL')
    mask2 = lMask.eq(4).Or(lMask.eq(5)).rename('landmask')
    return image.addBands([mask, mask2])
aa = getqa60LandMask2(badImage)
Map.addLayer(aa.select('landmask'))
bb = pctCloudLand(aa)
lpct = bb.get('lPct')
print(lpct.getInfo())

cpct = bb.get('cPct')
print(cpct.getInfo())

5
21.49476679927675


In [22]:
palette = ['111149','cdb33b','369b47']
Map.addLayer(s2_col.first().select('QA60'), {'min': 0, 'max': 2048, 'palette': palette}, 'clouds')

vizParams = {
  'bands': ['B8', 'B4', 'B3'],
  'min': 188,
  'max': 4103,
  'gamma': 1
}

Map.addLayer(s2_col.first(), vizParams, 'land')

In [15]:
ac = ee.ImageCollection('COPERNICUS/S2_SR').filterDate('2019-01-01','2020-01-01')
vizParams = {
  'bands': ['B8', 'B4', 'B3'],
  'min': 188,
  'max': 4103,
  'gamma': 1
}
Map.addLayer(ac.first(), vizParams, 'aa')


aa = getqa60LandMask(ac.first())


bb = pctCloudLand(aa)

bandNames = bb.bandNames()
print('Band names: ', bandNames.getInfo())
palette = ['999999', 'f781bf', 'a65628', 'ffff33', 'ff7f00', '984ea3', '4daf4a', '377eb8', 'e41a1c']
Map.addLayer(bb.select('SCL'), {'min':0, 'max':20, 'palette': palette}, 'scl')


Band names:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60', 'qa60mask', 'landmask']


In [28]:
properties = aa.propertyNames()
print('Metadata properties: ', properties.getInfo())

Metadata properties:  ['DATATAKE_IDENTIFIER', 'AOT_RETRIEVAL_ACCURACY', 'CLOUD_COVERAGE_PERCENTAGE', 'SPACECRAFT_NAME', 'FORMAT_CORRECTNESS_FLAG', 'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE', 'system:id', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A', 'CLOUD_SHADOW_PERCENTAGE', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'system:footprint', 'VEGETATION_PERCENTAGE', 'SOLAR_IRRADIANCE_B12', 'system:version', 'SOLAR_IRRADIANCE_B10', 'SOLAR_IRRADIANCE_B11', 'GENERATION_TIME', 'SOLAR_IRRADIANCE_B8A', 'SENSOR_QUALITY_FLAG', 'CLOUD_COVERAGE_ASSESSMENT', 'THIN_CIRRUS_PERCENTAGE', 'system:time_end', 'WATER_VAPOUR_RETRIEVAL_ACCURACY', 'system:time_start', 'DATASTRIP_ID', 'PROCESSING_BASELINE', 'SENSING_ORBIT_NUMBER', 'GEOMETRIC_QUALITY_FLAG', 'NODATA_PIXEL_PERCENTAGE', 'SENSING_ORBIT_DIRECTION', 'GRANULE_ID', 'LOW_PROBA_CLOUDS_PERCENTAGE', 'REFLECTANCE_CONVERSION_CORRECTION', 'MEDIUM_PROBA_CLOUDS_PERCENTAGE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8', 'DATATAKE_TYPE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6

In [16]:
lpct = bb.get('lPct')
print(lpct.getInfo())

cpct = bb.get('cPct')
print(cpct.getInfo())



0
0


In [24]:
vizParams = {
  'bands': ['B8', 'B4', 'B3'],
  'min': 188,
  'max': 4103,
  'gamma': 1
}
Map.addLayer(aa, vizParams, 'aa')

In [29]:
vp = {
  'bands': ['SWIR_1', 'NIR', 'RED'],
  'min': 242,
  'max': 4180,
  'gamma': 1
}

ids = range(0,144)
a = 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_'
b = '_2019_2020'
ilist = list()
[ilist.append(a+str(m)+b) for m in ids if m != 63]
    
print(ilist)
ic = ee.ImageCollection(ilist)
Map.addLayer(ic.mosaic(), vp, 'meanMosaic')

['projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_0_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_1_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_2_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_3_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_4_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_5_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_6_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_7_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_8_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_9_2019_2020', 'projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_1

In [17]:
Map.addLayer(ee.Image('projects/ee-jantzenator/assets/sentinel_acorr/SENT2_MEAN_COLOMBIA/Sent2_mean_0_2019_2020'))

In [32]:
230-144

86